# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import math
from joblib import Parallel, delayed
import numpy as np
import networkx as nx
import gensim

class MetricWalk:
    def __init__(self, graph: nx.Graph, dimensions: int = 128, window: int = 10, 
                 walk_length: int = 80, num_walks: int = 10, 
                 workers: int = 1):
        self.graph = graph
        self.dimensions = dimensions
        self.window = window
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.workers = workers

    def jaccard_coefficient(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            denominator = len(curr)+len(dest)
            if denominator == 0:
                score = 0
            else:
                numerator = len(set(curr+dest))
                score = numerator/denominator
        else:
            score = 0
        return score 

    def adamic_adar(self, current_node, destination):
        G = self.graph
        num_jobs = self.workers
        def calc_weight(node):
            denominator = math.log10(len(list(G.neighbors(node))))
            if denominator == 0:
                return 0
            else:
                return 1 / denominator
            
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            intersection = set(curr+dest)
            
            adamic_adar_indices = sum(Parallel(n_jobs=num_jobs)(
                delayed(calc_weight)(node) for node in intersection
            ))
            
            score = adamic_adar_indices
        else:
            score = 0
        return score

    def common_neighbors(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            score = len(set(curr+dest))
        else:
            score = 0
        return score 

    def lhn_index(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            denominator = len(curr)*len(dest)
            if denominator == 0:
                score = 0
            else:
                numerator = len(set(curr+dest))
                score = numerator/denominator
        else:
            score = 0
        return score 

    def preferential_attachment(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            score = len(curr) * len(dest)
        else:
            score = 0
        return score 

    def hub_promoted(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            denominator = min(len(curr),len(dest))
            if denominator == 0:
                score = 0
            else:
                numerator = len(set(curr+dest))
                score = numerator/denominator
        else:
            score = 0
        return score 

    def hub_depressed(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            denominator = max(len(curr),len(dest))
            if denominator == 0:
                score = 0
            else:
                numerator = len(set(curr+dest))
                score = numerator/denominator
        else:
            score = 0
        return score 

    def salton_index(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            denominator = math.sqrt(len(curr)*len(dest))
            if denominator == 0:
                score = 0
            else:
                numerator = len(set(curr+dest))
                score = numerator/denominator
        else:
            score = 0
        return score 

    def sorenson_index(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            denominator = len(curr)+len(dest)
            if denominator == 0:
                score = 0
            else:
                numerator = 2*len(set(curr+dest))
                score = numerator/denominator
        else:
            score = 0
        return score 

    def resource_allocation(self, current_node, destination):
        G = self.graph
        num_jobs = self.workers
        def calc_weight(node):
            denominator = len(list(G.neighbors(node)))
            if denominator == 0:
                return 0
            else:
                return 1 / denominator
            
        if current_node != destination:
            curr = list(G.neighbors(current_node))
            dest = list(G.neighbors(destination))
            intersection = set(curr+dest)
            
            resource_allocation_indices = sum(Parallel(n_jobs=num_jobs)(
                delayed(calc_weight)(node) for node in intersection
            ))
            
            score = resource_allocation_indices
        else:
            score = 0
        return score

    def max_flow(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            score, _ = nx.maximum_flow(G, current_node, destination)
        else:
            score = 0
        return score

    def min_cost_max_flow(self, current_node, destination):
        G = self.graph
        if current_node != destination:
            mincostFlow = nx.max_flow_min_cost(G, current_node, destination)
            score = nx.cost_of_flow(G, mincostFlow)
        else:
            score = 0
        return score 

    def generate_transition(self, transition_matrix_function):
        G = self.graph
        nodes = G.nodes
        num_workers = self.workers
        # Split the nodes into chunks for each worker
        node_chunks = np.array_split(nodes, num_workers)

        # Use joblib to parallelize the calculation of ss_weight
        scores = Parallel(n_jobs=num_workers)(
            delayed(transition_matrix_function)(current_node, destination)
            for chunk in node_chunks
            for current_node in chunk
            for destination in nodes
        )

        # Reshape the unnormalized scores into an unnormalized transition probability matrix
        unnormalized_transition = np.reshape(scores, (len(nodes), len(nodes)))


        # check if row sums are zero
        row_sums = unnormalized_transition.sum(axis=1)
        zero_rows = np.where(row_sums == 0)[0]

        # set all elements in zero rows to zero, except for diagonal element
        unnormalized_transition[zero_rows, :] = 0
        for i in zero_rows:
            unnormalized_transition[i, i] = 1

        row_sums = unnormalized_transition.sum(axis=1)

        # normalize matrix by row sums
        normalized_transition = np.divide(unnormalized_transition, row_sums[:, np.newaxis])
        return normalized_transition

    def generate_walks(self, transition_probs):
        graph = self.graph
        num_walks = self.num_walks
        walk_length = self.walk_length
        num_jobs = self.workers 
        walks = []
        nodes = list(graph.nodes())

        # Convert the transition probabilities to a dictionary of dictionaries for faster access
        probs = {}
        for i, node_i in enumerate(nodes):
            probs[node_i] = {}
            for j, node_j in enumerate(nodes):
                probs[node_i][node_j] = transition_probs[i][j]

        def generate_walks_for_node(node):
            node_walks = []
            for walk in range(num_walks):
                walk_list = [node]
                for step in range(walk_length - 1):
                    neighbors = list(probs[walk_list[-1]].keys())
                    probabilities = list(probs[walk_list[-1]].values())
                    next_node = np.random.choice(neighbors, p=probabilities)
                    walk_list.append(next_node)
                node_walks.append(walk_list)
            return node_walks

        node_walks_list = Parallel(n_jobs=num_jobs)(
            delayed(generate_walks_for_node)(node) for node in nodes)

        for node_walks in node_walks_list:
            walks += node_walks

        return walks

    def fit(self, transformation: callable) -> gensim.models.Word2Vec:
        transition_probability_matrix = self.generate_transition(transformation)
        walks = self.generate_walks(transition_probability_matrix)
        return gensim.models.Word2Vec(walks,window=self.window, workers=self.workers, vector_size=self.dimensions)
